# Ejmplo de RAG - Retrival Augmnted Generation

In [ ]:
#!pip install python-dotenv

In [ ]:
#!pip install weaviate-client==4.5.4

In [ ]:
#!pip install langchain==0.1.16

In [ ]:
#!pip install sentence-transformers==2.6.1

In [ ]:
#!pip install google-generativeai==0.5.2

In [ ]:
#!pip uninstall protobuf -y

In [ ]:
#!pip install protobuf==4.25.7

### Dependencias usadas para que corra sin problema:

In [ ]:
#- pip:
#      - aiohappyeyeballs==2.6.1       - aiohttp==3.11.18      - aiosignal==1.3.2       - annotated-types==0.7.0       - authlib==1.5.2
#      - cachetools==5.5.2       - cryptography==45.0.2       - dataclasses-json==0.6.7       - filelock==3.18.0
#      - frozenlist==1.6.0       - fsspec==2025.5.0       - google-ai-generativelanguage==0.6.2       - google-api-core==2.25.0rc1
#      - google-api-python-client==2.169.0       - google-auth==2.40.1       - google-auth-httplib2==0.2.0       - google-generativeai==0.5.2
#      - googleapis-common-protos==1.70.0       - greenlet==3.2.2       - grpcio==1.71.0       - grpcio-health-checking==1.62.3
#      - grpcio-status==1.62.3      - grpcio-tools==1.62.3       - httplib2==0.22.0       - httpx==0.27.0
#      - huggingface-hub==0.31.4       - joblib==1.5.0       - jsonpatch==1.33       - jsonpointer==3.0.0
#      - langchain==0.1.16       - langchain-community==0.0.38       - langchain-core==0.1.53       - langchain-text-splitters==0.0.2
#      - langsmith==0.1.147       - marshmallow==3.26.1       - mpmath==1.3.0       - multidict==6.4.4
#      - mypy-extensions==1.1.0       - networkx==3.4.2       - numpy==1.26.4       - orjson==3.10.18
#      - packaging==23.2       - pillow==11.2.1       - propcache==0.3.1       - proto-plus==1.26.1
#      - protobuf==4.25.7       - pyasn1==0.6.1       - pyasn1-modules==0.4.2       - pydantic==2.11.4
#      - pydantic-core==2.33.2       - pyparsing==3.2.3       - python-dotenv==1.1.0       - regex==2024.11.6
#      - requests-toolbelt==1.0.0       - rsa==4.9.1       - safetensors==0.5.3       - scikit-learn==1.6.1
#      - scipy==1.15.3       - sentence-transformers==2.6.1       - sqlalchemy==2.0.41       - sympy==1.14.0
#      - tenacity==8.5.0       - threadpoolctl==3.6.0       - tokenizers==0.21.1       - torch==2.7.0
#      - tqdm==4.67.1       - transformers==4.52.2       - typing-inspect==0.9.0       - typing-inspection==0.4.0
#      - uritemplate==4.1.1       - validators==0.22.0       - weaviate-client==4.5.4       - yarl==1.20.0


## Se carga la configuración local del archivo .env que contiene las llaves de acceso a Geminai de google.

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
print(os.environ["GOOGLE_API_KEY"])

## Ejemplo de implementación de Embeddings opensource, sobre weaviate, con a IA gratuita de Google.

In [ ]:
import weaviate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import google.generativeai as genai

# Configuración de Gemini 
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
# 1. Cargar el libro

with open('La-Republilca-Platon.txt', encoding='utf-8') as f:
    libro = f.read()

# 2. Separar en chunks semánticos

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

chunks = splitter.split_text(libro)

print(f"Numero de pedazos: {len(chunks)}")
print(chunks[0])

# 3. Inicializar el modelo de embedding opensource
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# 4. Conectar a Weaviate local

client = weaviate.Client("http://localhost:8080")


# 5. Crear esquema de weaviate si no existe

if not client.schema.exists("LibroChunk"):
    client.schema.create_class({
        "class":"LibroChunk",
        "vectorizer":"none",
        "properties":[
            {"name":"texto","dataType":["text"]}
        ],
    })    

# 6. Indexar los chunks en Weaviate

for chunk in chunks:
    vector = embedder.encode(chunk).tolist()
    client.data_object.create(
        data_object = {"texto":chunk},
        class_name = "LibroChunk",
        vector=vector
    )

# 7. Funcion para consultar por tema usando embeddings y Gemini
def consulta_libro(pregunta):
    pregunta_vector = embedder.encode(pregunta).tolist()
    # Busca los tres chunks más relevantes
    result = client.query.get("LibroChunk",["texto"]).with_near_vector({"vector":pregunta_vector}).with_limit(3).do()
    textos = [d["texto"] for d in result["data"]["Get"]["LibroChunk"]]
    print("------------------------ TEXTOS -----------------------------")
    print(textos)
    print("-------------------------------------------------------------")
    contexto = "\n".join(textos)
    # Crear el modelo Gemini
    model = genai.GenerativeModel("gemini-2.0-flash")
#    respuesta = model.generate_content(
#        [
#            {"role":"system","parts":[{"text":"Responde usando solo el contexto proporcionado."}]},
#            {"role":"user","parts":[{"text":f"Contexto:\n{contexto}\n\nPregunta: {pregunta}"}]}
#        ]
    respuesta = model.generate_content(
        [
            {"role":"user","parts":[{"text":f"Responde solo usando el contexto proporcionado.\n\nContexto:\n{contexto}\n\nPregunta: {pregunta}"}]}
        ]
    )
    return respuesta.text

def elimina_esquema(esquema):
    client = weaviate.Client("http://localhost:8080")
    if client.schema.exists(esquema):
        client.schema.delete_class(esquema)
        print(f"Esquema {esquema} eliminado correctamente")
    else:
        print(f"El esquema {esquema} no existe.")

# 8. Ejemplo de pregunta
if __name__=="__main__":
    #elimina_esquema("LibroChunk")
    pregunta = "¿Qué dice el texto sobre gobernar?"
    print("Pregunta:", pregunta)
    print("Respuesta:",consulta_libro(pregunta))
    
    pregunta = "¿Qué dice el texto sobre la democracia?"
    print("Pregunta:", pregunta)
    print("Respuesta:",consulta_libro(pregunta))

    
    